
## LGBTI análisis

En esta notebook analizamos un poco los resultados sobre la categoría LGBTI. Acá podemos ver que ChatGPT anda sensiblemente mejor que BETO. 

¿Qué pasa? => recordemos que en este caso el recall es mucho mejor en los LLMS. Ver [Resultados](Results_full.ipynb)


In [1]:

%load_ext autoreload
%autoreload 2
import pandas as pd
from glob import glob
pd.options.display.max_colwidth = 1000
#df_train.loc[df_train["LGBTI"] > 0, ["title", "text", "text_label"]].sample(20)

files = {
    #"one-shot": sorted(glob("../data/test*pred*1shot.csv")),
    "gpt-3.5": sorted(glob("../data/runs/test_*_chatgpt.csv")),
    "beto": sorted(glob("../data/runs/test*beto.csv")),
    "aya": sorted(glob("../data/runs/test*aya.csv")),
    "mixtral": sorted(glob("../data/runs/test*mixtral.csv")),
}

#assert len({len(value) for value in files.values()}) == 1

dfs = {k: pd.concat([pd.read_csv(f, index_col=0) for f in v]) for k, v in files.items()}
{k: v.shape for k, v in dfs.items()}

{'gpt-3.5': (5670, 24),
 'beto': (5670, 22),
 'aya': (5670, 25),
 'mixtral': (5670, 25)}

In [2]:
from rioplatense_hs.preprocessing import text_to_label, labels
from sklearn.metrics import classification_report

pred_labels = [f"PRED_{label}" for label in labels]
score_labels = [f"SCORE_{label}" for label in labels]

dfs["beto"][score_labels] = dfs["beto"][pred_labels]
dfs["beto"][pred_labels] = (dfs["beto"][score_labels] > 0.5).astype(int)

dfs["beto"][score_labels + pred_labels]

,SCORE_WOMEN,SCORE_LGBTI,SCORE_RACISM,SCORE_CLASS,SCORE_POLITICS,SCORE_DISABLED,SCORE_APPEARANCE,SCORE_CRIMINAL,PRED_WOMEN,PRED_LGBTI,PRED_RACISM,PRED_CLASS,PRED_POLITICS,PRED_DISABLED,PRED_APPEARANCE,PRED_CRIMINAL
id,,,,,,,,,,,,,,,,
401109,0.001725,0.000662,0.000618,0.001230,0.206528,0.001362,0.002174,0.001290,0,0,0,0,0,0,0,0
396214,0.002799,0.002077,0.001808,0.000510,0.000687,0.000524,0.000969,0.000139,0,0,0,0,0,0,0,0
398849,0.000374,0.000365,0.000927,0.000313,0.000383,0.000293,0.000544,0.000334,0,0,0,0,0,0,0,0
398914,0.003084,0.384382,0.002267,0.001005,0.001925,0.002505,0.002260,0.001661,0,0,0,0,0,0,0,0
382020,0.004154,0.001073,0.042247,0.001076,0.001832,0.000817,0.000514,0.060930,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406274,0.001073,0.000493,0.000469,0.000309,0.000469,0.001115,0.001577,0.008345,0,0,0,0,0,0,0,0
395737,0.000410,0.000303,0.000655,0.000226,0.000440,0.000238,0.000515,0.000299,0,0,0,0,0,0,0,0
365805,0.000363,0.000282,0.001139,0.000197,0.000434,0.000272,0.000623,0.000243,0,0,0,0,0,0,0,0


In [3]:
from rioplatense_hs.preprocessing import labels as all_labels

labels = ["WOMEN", "LGBTI", "RACISM", "CLASS"]


## LGBTI 

BETO vs few-shot ChatGPT

In [4]:
dfs["gpt-3.5"].columns

Index(['title', 'text', 'context_tweet', 'HATEFUL', 'CALLS', 'WOMEN', 'LGBTI',
       'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL',
       'prompt', 'pred_cot', 'PRED_WOMEN', 'PRED_LGBTI', 'PRED_RACISM',
       'PRED_CLASS', 'PRED_POLITICS', 'PRED_DISABLED', 'PRED_APPEARANCE',
       'PRED_CRIMINAL', 'PRED_HATEFUL'],
      dtype='object')

In [5]:
dfs["beto"].columns

Index(['title', 'text', 'context_tweet', 'HATEFUL', 'CALLS', 'WOMEN', 'LGBTI',
       'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL',
       'PRED_CALLS', 'PRED_WOMEN', 'PRED_LGBTI', 'PRED_RACISM', 'PRED_CLASS',
       'PRED_POLITICS', 'PRED_DISABLED', 'PRED_APPEARANCE', 'PRED_CRIMINAL',
       'SCORE_WOMEN', 'SCORE_LGBTI', 'SCORE_RACISM', 'SCORE_CLASS',
       'SCORE_POLITICS', 'SCORE_DISABLED', 'SCORE_APPEARANCE',
       'SCORE_CRIMINAL'],
      dtype='object')

In [6]:
from rioplatense_hs.mixtral import post_process_output




dfs["mixtral"]["pred_cot"] = dfs["mixtral"]["pred_cot"].apply(post_process_output)

In [7]:
# Merge dfs['few-shot'] and dfs['beto']

columns_to_drop = [
    "title", "text", "context_tweet",
    # Remove gold labels
    "HATEFUL", "CALLS", "WOMEN", "LGBTI", "RACISM", "CLASS", "POLITICS", "DISABLED", "CRIMINAL", "APPEARANCE",
    # Remove predictions of dismissed labels
    "PRED_CALLS", "PRED_POLITICS", "PRED_DISABLED", "PRED_CRIMINAL", "PRED_APPEARANCE",
]

base_df = dfs["gpt-3.5"].copy()

base_df.drop([
    'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL',
    'PRED_POLITICS',
    'PRED_DISABLED', 'PRED_APPEARANCE', 'PRED_CRIMINAL', 'PRED_HATEFUL'
], axis=1, inplace=True)

print(base_df.columns)
df_join = base_df.join(dfs["beto"].drop(columns_to_drop, axis=1), rsuffix="_beto", lsuffix="_gpt")

mixtral = dfs["mixtral"].copy()

mixtral.drop([c for c in mixtral.columns if c in columns_to_drop], axis=1, inplace=True)
mixtral.rename(columns={f"PRED_{label}": f"PRED_{label}_mixtral" for label in all_labels}, inplace=True)
#mixtral.rename(columns={"pred_cot": "pred_cot_mixtral", "prompt": "prompt_mixtral"}, inplace=True)

df_join = df_join.join(mixtral, rsuffix="_mixtral", lsuffix="_gpt")

cols = [c for c in df_join.columns if c.isupper() and "SCORE" not in c]
#
df_join[cols] = df_join[cols].astype(bool)

sorted(df_join.columns)

Index(['title', 'text', 'context_tweet', 'HATEFUL', 'CALLS', 'WOMEN', 'LGBTI',
       'RACISM', 'CLASS', 'prompt', 'pred_cot', 'PRED_WOMEN', 'PRED_LGBTI',
       'PRED_RACISM', 'PRED_CLASS'],
      dtype='object')


['CALLS',
 'CLASS',
 'HATEFUL',
 'LGBTI',
 'PRED_CLASS_beto',
 'PRED_CLASS_gpt',
 'PRED_CLASS_mixtral',
 'PRED_HATEFUL',
 'PRED_LGBTI_beto',
 'PRED_LGBTI_gpt',
 'PRED_LGBTI_mixtral',
 'PRED_RACISM_beto',
 'PRED_RACISM_gpt',
 'PRED_RACISM_mixtral',
 'PRED_WOMEN_beto',
 'PRED_WOMEN_gpt',
 'PRED_WOMEN_mixtral',
 'RACISM',
 'SCORE_APPEARANCE',
 'SCORE_CLASS',
 'SCORE_CRIMINAL',
 'SCORE_DISABLED',
 'SCORE_LGBTI',
 'SCORE_POLITICS',
 'SCORE_RACISM',
 'SCORE_WOMEN',
 'WOMEN',
 'context_tweet',
 'error',
 'pred_cot_gpt',
 'pred_cot_mixtral',
 'prompt_gpt',
 'prompt_mixtral',
 'text',
 'title']

Ejemplos que falla BETO y acierta ChatGPT y eran LGBTI

In [8]:

beto_false_positives_detected_by_chatgpt = df_join.loc[
    df_join["LGBTI"] & df_join["PRED_LGBTI_gpt"] & ~df_join["PRED_LGBTI_beto"]
]

columns_to_show = ["context_tweet", "text", "pred_cot_gpt", "pred_cot_mixtral", "SCORE_LGBTI"]

beto_false_positives_detected_by_chatgpt[columns_to_show]

,context_tweet,text,pred_cot_gpt,pred_cot_mixtral,SCORE_LGBTI
id,,,,,
398914,Mara Gómez: la historia de la primera futbolista trans en el torneo argentino https://t.co/rBx3pMPUbB https://t.co/l5I1j0xFku,"@usuario Ponga huevos, Mara ponga huevos...","El texto alienta a Mara Gómez como futbolista, pero utiliza la expresión ""ponga huevos"" de manera discriminatoria, con el objetivo de cuestionar su masculinidad y su identidad de género como mujer trans. La respuesta final es ""lgbti"".","El texto alude a la falta de ""huevos"" de Mara Gómez, un término que se utiliza para referirse a la falta de coraje o valentía. Esto es ofensivo y despectivo. La respuesta final es ""mujer, lgbti"".",0.384382
405355,Tras una ráfaga de más de 20 disparos asesinaron a una mujer trans en Rosario https://t.co/M0K3NXgfA8,@usuario En la autopsia no debe figurar la próstata por ley! 😄,"El mensaje hace referencia a la identidad de género de la mujer trans asesinada de manera discriminatoria, utilizando un emoji para burlarse de la situación. La respuesta final es ""lgbti"".","El texto se refiere a la víctima como ""un hombre"" y niega su identidad de género. La respuesta final es ""lgbti"".",0.061692
359386,Eugenio Zaffaroni: “Con Macri no hubiéramos tenido cuarentena y tendríamos miles de muertos” https://t.co/DhKGcEjTzz https://t.co/0XURGdhX3C,@usuario VOS ZAFARONI HDP CUIDATE Y NO HABLES MAS PUTO DE MIERDA,"El texto contiene insultos homofóbicos hacia Eugenio Zaffaroni (""puto"") y deseo de daño físico (""cuidate y no hables más""). Por lo tanto, el discurso de odio presente es ""lgbti"".","El texto insulta a Eugenio Zaffaroni con palabras despectivas y homofóbicas. La respuesta final es ""mujer, lgbti"".",0.000754
382137,"Elizabeth Gómez Alcorta: “Por la pandemia, vamos a tener una suba de los femicidios y travesticidios” | Por Luciana Peker https://t.co/OGEL4INojA",@usuario Trasvescidios? 🤔\n\n Por la cercanía del verano vamos a ver un aumento en la tasa de los mosquicidios,"El texto hace una burla despectiva con el término ""travesticidios"", minimizando la problemática de los crímenes de odio contra personas trans. Aunque no menciona directamente a personas de la comunidad lgbti, la comparación con ""mosquicidios"" es discriminatoria. La respuesta final es ""lgbti"".","El texto se burla de la identidad de género de las personas trans y de la palabra ""travesticidios"". La respuesta final es ""lgbti"".",0.000728
395663,"Alberto Fernández: ""Me iría a un isla desierta con Máximo Kirchner "" https://t.co/J11MNnD5Yb https://t.co/ALGYab7Xch",@usuario @usuario Si el gordo tiene una pinta de sodomita!JAJA,"El texto utiliza un término despectivo para referirse a la orientación sexual de Máximo Kirchner (""sodomita""), lo cual constituye discurso de odio hacia la comunidad LGBTI. La respuesta final es ""lgbti"".","El texto se refiere al presidente Alberto Fernández con un insulto homofóbico. La respuesta final es ""lgbti"".",0.026260
396716,"""Te amo "". La emotiva dedicatoria de Luis Novaresio a su pareja en su cumpleaños https://t.co/Hh63kaVA5U https://t.co/vNaUBjBkzx",@usuario Definitivamente no acepto esta degeneración repugnante de la humanidad.,"El texto expresa un rechazo hacia la relación homosexual de Luis Novaresio, ya que utiliza términos como ""degeneración repugnante"". Por lo tanto, el tweet contiene un discurso de odio hacia la comunidad LGBTI. La respuesta final es ""lgbti"".","El texto muestra un rechazo hacia la homosexualidad de Luis Novaresio, llamándola ""degeneración repugnante"". La respuesta final es ""lgbti"".",0.065278
405366,Tras una ráfaga de más de 20 disparos asesinaron a una mujer trans en Rosario https://t.co/M0K3NXgfA8,@usuario Salió de caño... cuac!,"El texto utiliza un término despectivo (""caño"") para referirse a la forma en que asesinaron a la mujer trans. Aunque no insulta directamente a la mujer trans, la forma en que se refiere a su muerte es despectiva y minimiza la gravedad del crimen. La respuesta final es ""lg

In [9]:

not_detected_by_any = df_join.loc[
    df_join["LGBTI"] & ~df_join["PRED_LGBTI_gpt"] & ~df_join["PRED_LGBTI_beto"]
]

not_detected_by_any[columns_to_show]

,context_tweet,text,pred_cot_gpt,pred_cot_mixtral,SCORE_LGBTI
id,,,,,
396723,"""Te amo "". La emotiva dedicatoria de Luis Novaresio a su pareja en su cumpleaños https://t.co/Hh63kaVA5U https://t.co/vNaUBjBkzx",@usuario Asco dan.,"El texto no menciona ninguna característica protegida ni contiene discurso de odio. La respuesta final es ""nada"".","El texto muestra un rechazo hacia la homosexualidad de Luis Novaresio y su pareja. La respuesta final es ""lgbti"".",0.082625
380747,😱🔥[¡INESPERADO!]😱🔥 Luis Novaresio recomendó un juguete sexual en las redes &gt;&gt;&gt; El periodista compartió en sus redes sociales una peculiar noticia sobre un objeto para el placer masculino que se volvió viral. ¡Entrá a la nota y mirá! https://t.co/U8NuqEKeQX,"@usuario Se cree ""periodista""... y es una vedetonga","El texto desprecia la profesión de periodista de Luis Novaresio y lo compara con una vedette de manera despectiva. Aunque se refiere a una característica protegida (mujer), no contiene discurso de odio. La respuesta final es ""nada"".","El texto desprecia a Luis Novaresio por su supuesta falta de profesionalismo y lo compara con una vedette, lo que implica una falta de respeto hacia su persona. Sin embargo, no hace ninguna alusión a una característica protegida. La respuesta final es 'nada'.",0.007522
338106,Vive con sus dos novios y atiende “la policlínica” del amor libre: 5 preguntas para abrir una relación y sobrevivir al intento | Por Gisele Sousa Dias https://t.co/pBZLnGiDa2,@usuario Terrible porquería debe ser su vida,"El texto expresa desagrado hacia la forma de vida de la persona, pero no contiene discurso de odio ni menciona alguna característica protegida. La respuesta final es ""nada"".","El texto desprecia la vida de la persona mencionada, sin hacer referencia a ninguna característica protegida. La respuesta final es 'nada'.",0.007555
379538,“¿Por qué no vemos médicos trans?”: el reclamo de un prestigioso cardiólogo para que América sea más inclusiva | Por Valeria Román https://t.co/19put40QqT,"@usuario Raro que no lleguen a ser médicos o ingenieros, son carreras duras y difíciles.\nPero nadie mejor que ellos sabe lo que es romperse el culo...","El texto contiene un comentario que podría interpretarse como una descalificación hacia las personas trans al insinuar que es sorprendente que logren ser médicos o ingenieros, ya que son carreras difíciles. Sin embargo, al final el texto intenta ser elogioso al mencionar que nadie mejor que ellos sabe lo que es ""romperse el culo"", refiriéndose al esfuerzo necesario para alcanzar el éxito. A pesar de la inicial descalificación, el comentario logra ser más positivo al final. La respuesta final es ""nada"".","El texto muestra estereotipos sobre las carreras que los miembros de la comunidad LGBTQ+ pueden elegir, sugiriendo que no son capaces de realizar trabajos difíciles. La respuesta final es ""lgbti"".",0.202822
398934,Mara Gómez: la historia de la primera futbolista trans en el torneo argentino https://t.co/rBx3pMPUbB https://t.co/l5I1j0xFku,@usuario Viene a traer Lo que le falta al equipo: goles y mucho huevo,"El texto elogia las habilidades futbolísticas de Mara Gómez y menciona que planea aportar goles y esfuerzo al equipo. No contiene discurso de odio ni menciona ninguna característica protegida. La respuesta final es ""nada"".","El texto alude a la capacidad de Mara Gómez como futbolista, sin hacer ninguna alusión a una característica protegida. La respuesta final es 'nada'.",0.323285
396730,"""Te amo "". La emotiva dedicatoria de Luis Novaresio a su pareja en su cumpleaños https://t.co/Hh63kaVA5U https://t.co/vNaUBjBkzx",@usuario 🙈🙈🙈🙊🤮,"El texto no contiene discurso de odio, ya que no hace referencia a ninguna característica protegida ni contiene insultos dirigidos a un individuo o grupo en particular. La respuesta final es ""nada"".",El texto no contiene discurso de odio. La respuesta final es 'nada'.,0.016485
390770,″¿Por qué un beso entre dos hombres los vuelve tan violentos

In [10]:

mixtral_errors = df_join.loc[
    ~df_join["LGBTI"] & ~df_join["PRED_LGBTI_gpt"] & ~df_join["PRED_LGBTI_beto"] & df_join["PRED_LGBTI_mixtral"]
]

mixtral_errors[["context_tweet", "text", "pred_cot_mixtral", "pred_cot_gpt", "SCORE_LGBTI"]].sample(10)

,context_tweet,text,pred_cot_mixtral,pred_cot_gpt,SCORE_LGBTI
id,,,,,
395345,"El exabrupto de Jair Bolsonaro sobre Brasil y el Covid-19: ""Tienen que dejar de ser un país de maricas"" https://t.co/WtxRjY7tJp",@usuario Excelentes declaraciones,"El texto alaba las declaraciones de Bolsonaro, que incluyen un insulto homofóbico. La respuesta final es ""lgbti"".","El tweet no contiene discurso de odio, a pesar de que Bolsonaro utiliza un lenguaje despectivo al referirse a Brasil como ""un país de maricas"". La respuesta final es ""nada"".",0.001061
369460,"Marcelo Tinelli, entre rumores de romance con Laurita Fernández y su reconciliación con Guillermina Valdés https://t.co/WX23HPycpd","@usuario Reconciliate con el Cholo Simeone, Marce. Salí del armario.","El texto sugiere que Marcelo Tinelli es homosexual y que debe salir del armario. Esto es una forma de burla hacia la comunidad LGBTQ+. La respuesta final es ""lgbti"".","El texto no contiene discurso de odio, ya que aunque menciona la posibilidad de que Marcelo Tinelli tenga una relación romántica con un hombre, no lo hace de manera despectiva ni discriminativa. La respuesta final es ""nada"".",0.000555
379457,El PAMI tendrá que pagar una cirugía para agrandar los pechos de una afiliada autopercibida mujer | Por Patricia Blanco https://t.co/IObp8ZBnxq,@usuario Y los jubilados se mueren de hambre. Esta boludez votaron.,"El texto critica la decisión del PAMI de pagar una cirugía de aumento de senos a una afiliada trans. La respuesta final es ""lgbti"".","El tweet no contiene discurso de odio ya que, si bien critica la decisión de financiar una cirugía estética, no realiza ninguna declaración de carácter intenso o irracional de rechazo hacia un individuo o grupo por una característica protegida. La respuesta final es ""nada"".",0.000872
390775,″¿Por qué un beso entre dos hombres los vuelve tan violentos?”: la vida después de haber sido víctima de ataques homofóbicos | Por Milton Del Moral https://t.co/MgHNMf8Sch,@usuario @usuario No estamos acostumbrados!!,"El texto argumenta que la sociedad no está acostumbrada a ver besos entre dos hombres, lo que implica un rechazo a la homosexualidad. La respuesta final es ""lgbti"".","El texto no contiene discursos de odio, ya que si bien expresa una postura de sorpresa o desaprobación, no se refiere de manera ofensiva a ninguna característica protegida. La respuesta final es ""nada"".",0.001107
359561,Polémica en las redes por el actor que personificó a Manuel Belgrano y habló en lenguaje inclusivo https://t.co/zp904FUkPj,@usuario Pobre Argentina.,"El texto muestra un desprecio hacia el uso del lenguaje inclusivo. Aunque no hace una alusión directa a una característica protegida, el lenguaje inclusivo es una herramienta utilizada por el movimiento LGBT+. La respuesta final es ""lgbti"".","El texto no contiene discurso de odio. No se menciona ninguna característica protegida ni se expresa rechazo hacia algún grupo en particular. La respuesta final es ""nada"".",0.000821
397509,Así luce el actor Elliot Page tras declararse trans https://t.co/yeTGhw6ooT,@usuario Exactamente igual que antes,El texto sugiere que no hay diferencia entre la apariencia de Elliot Page antes y después de declararse trans. Esto niega la identidad de género de Elliot Page y es ofensivo. La respuesta final es 'lgbti'.,"El texto no contiene discurso de odio, ya que simplemente menciona cómo luce Elliot Page tras declararse trans. No hay ninguna declaración de rechazo, enemistad o aborrecimiento contra el actor por su identidad de género. La respuesta final es ""nada"".",0.002553
398898,Mara Gómez: la historia de la primera futbolista trans en el torneo argentino https://t.co/rBx3pMPUbB https://t.co/l5I1j0xFku,"@usuario Cuando fracasaste en tu sueño de ser jugador profesional de Futbol, y t queda una última carta por jugar...","El texto desprecia a Mara Gómez por ser trans y por haber tenido que cambiar de camino en su carrera deportiva. La respuesta final es ""lgbti"".","El

In [16]:
# Show those which have one of these
# - Labeled as LGBTI
# - Detected by chatgpt as LGBTI
# - Detected by BETO as LGBTI
# - Detected by Mixtral

columns = [
    "context_tweet", "text", "LGBTI",
    "PRED_LGBTI_gpt", "PRED_LGBTI_beto", "PRED_LGBTI_mixtral",
    "pred_cot_gpt", "pred_cot_mixtral",
]
df_join.loc[
    df_join["LGBTI"] | df_join["PRED_LGBTI_gpt"] | df_join["PRED_LGBTI_beto"] | df_join["PRED_LGBTI_mixtral"],
    columns
].to_csv("../data/tables/lgbti_analysis.csv")